In [33]:
import json
import numpy as np

In [34]:
def linear(x):
    return x

def derivative_linear(x):
    return np.ones(x.shape)

def relu(x):
    return np.maximum(0, x)

def derivative_relu(x):
    return np.where(x < 0, 0, 1)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def derivative_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

def derivative_softmax(x, isTarget, idx):
    if isTarget:
        return softmax(x)[idx]
    else:
        return -(1-(softmax(x)[idx]))

In [35]:
def delta_output(activation_function, target, output, net, idx = 0):
  if activation_function == "linear":
      result = (target - output) * derivative_linear(net)
  elif activation_function == "relu":
      result = (target - output) * derivative_relu(net)
  elif activation_function == "sigmoid":
      result = (target - output) * derivative_sigmoid(net)
  elif activation_function == "softmax":
      result = -derivative_softmax(net, True, idx)

  return result

def delta_hidden(activation_function, net, delta_output, weights, idx = 0):
  if activation_function == "linear":
        delta_output_np = np.array(delta_output)
        weights_np = np.array(weights.T)
        result = np.sum(-np.dot(delta_output_np, weights_np) * derivative_linear(net))
  elif activation_function == "relu":
        delta_output_np = np.array(delta_output)
        weights_np = np.array(weights.T)
        result = np.sum(-np.dot(delta_output_np, weights_np) * derivative_relu(net))
  elif activation_function == "sigmoid":
        delta_output_np = np.array(delta_output)
        weights_np = np.array(weights.T)
        result = np.sum(-np.dot(delta_output_np, weights_np) * derivative_sigmoid(net))
  elif activation_function == "softmax":
      result = -derivative_softmax(net, False, idx)

  return result

def calculate_delta_weight(activation_function, learning_rate, target, output, net, weights, x, isOutputLayer, idx = 0, out = 1):
    if isOutputLayer:
        return learning_rate * delta_output(activation_function, target, output, net, idx) * x
    else:
        return learning_rate * delta_hidden(activation_function, net, out, weights) * x

In [36]:
def loss_function_sse(target, output):
    e = 0.5 * np.sum((target - output) ** 2)
    return e

def loss_function_softmax(target, output):
    e = -np.sum(target * np.log(output))
    return e

In [37]:
def terminate_condition(stopped_by, max_iteration, error_threshold, iteration, error):
    if stopped_by == "max_iteration":
        return iteration >= max_iteration
    elif stopped_by == "error_threshold":
        return error <= error_threshold

def forward_propagation(model, input_data, weights):
    layers = model["layers"]
    output_layer = [input_data]

    for i, layer in enumerate(layers):
        activation_function = layer["activation_function"]
        weight_matrix = weights[i]
        bias = layer.get("bias", 1)

        input_activation = output_layer[-1]
        # print("Dimensi input_activation:", input_activation.shape)
        # print(input_activation)
        if len(input_activation.shape) == 1:
            input_activation_with_bias = np.insert(input_activation, 0, bias)
            input_activation_with_bias = input_activation_with_bias.reshape(1, -1)
        else:
            batch_size = input_activation.shape[0]
            bias_vector = np.ones((batch_size, 1)) * bias
            input_activation_with_bias = np.concatenate((bias_vector, input_activation), axis=1)
        # print("Dimensi input_activation_with_bias:", input_activation_with_bias.shape)
        # print(input_activation_with_bias)
        # print("Dimensi weight_matrix:", weight_matrix.shape)
        # print(weight_matrix)

        output_linear_combination = np.dot(input_activation_with_bias, weight_matrix)

        if activation_function == "linear":
            activation_result = linear(output_linear_combination)
        elif activation_function == "relu":
            activation_result = relu(output_linear_combination)
        elif activation_function == "sigmoid":
            activation_result = sigmoid(output_linear_combination)
        elif activation_function == "softmax":
            activation_result = softmax(output_linear_combination)

        output_layer.append(activation_result)

    return output_layer

def back_propagation(model, input, initial_weights, target_input, learning_rate, batch_size, stopped_by, max_iteration, error_threshold):
    epoch=0
    error = np.inf

    while not terminate_condition(stopped_by, max_iteration, error_threshold, epoch, error) :
        epoch += 1

        if error == np.inf:
            error = 0

        for batch in range(int(input.shape[0]/batch_size)):
            input_batch = input[batch*batch_size:(batch+1)*batch_size]
            target_batch = target_input[batch*batch_size:(batch+1)*batch_size]
            output_forward = forward_propagation(model, input_batch, initial_weights)

            delta_weight = [np.zeros(w.shape) for w in initial_weights]
            for id_data in range(batch_size):
                # Calculate error each output unit
                error_k = []
                error_k_divided = []

                id = id_data + batch_size * batch

                if (model["layers"][-1]["activation_function"] == "softmax"):
                    all_net = []
                    # Iterate through each output unit
                    for i in range(len(output_forward[-1][id_data])):
                        input_x = output_forward[-2][id_data]
                        input_x = np.insert(input_x, 0, 1)
                        
                        all_net.append(np.dot(input_x.T, initial_weights[-1][:,i]))
                
                
                # Iterate through each output unit
                for i in range(len(output_forward[-1][id_data])):
                    activation_function = model["layers"][-1]["activation_function"]
                    output = output_forward[-1][id_data][i]
                    target = target_batch[id_data][i]
                    
                    input_x = output_forward[-2][id_data]
                    input_x = np.insert(input_x, 0, 1)
                    
                    weights = initial_weights[-1]
                    error_k_j = []
                    
                    if(activation_function != "softmax"):
                        net = np.dot(input_x.T, initial_weights[-1][:,i])

                    for j in range(input_x.shape[0]):
                        x = input_x[j]
                        
                        if (activation_function == "softmax"):
                            error_k_j_one_unit = calculate_delta_weight(activation_function, learning_rate, target, output, all_net, weights, x, True, i)
                            error_k_j_one_unit_divided = error_k_j_one_unit/(x * learning_rate)
                            error_k_j.append(error_k_j_one_unit)
                        else:
                            error_k_j_one_unit = calculate_delta_weight(activation_function, learning_rate, target, output, net, weights, x, True)
                            error_k_j_one_unit_divided = error_k_j_one_unit/(x * learning_rate)
                            error_k_j.append(error_k_j_one_unit)
                    error_k.append(error_k_j)
                    error_k_divided.append(error_k_j_one_unit_divided)

                    # if (activation_function == "softmax"): break
                
                        
                # Calculate error each hidden unit
                error_h = []
                
                # Iterate through each hidden layer
                for j in range(len(output_forward)-2, 0, -1):
                    error_h_j = []
                    error_h_j_divided = []
                    activation_function = model["layers"][j - 1]["activation_function"]

                    # Iterate through each hidden layer unit
                    for k in range(len(output_forward[j][id_data])):
                        output = output_forward[j][id_data][k]
                        target = target_batch[id_data][k]
                        
                        input_x = output_forward[j-1][id_data]
                        input_x = np.insert(input_x, 0, 1)
                        
                        net = np.dot(input_x.T, initial_weights[j - 1][:,k])

                        weights = initial_weights[j - 1]

                        error_h_j_k = []
                        for l in range(input_x.shape[0]):
                            x = input_x[l]
                            
                            if (activation_function == "softmax"):
                                error_h_j_k_one_unit = calculate_delta_weight(activation_function, learning_rate, target, output, all_net, weights, x, False)
                                error_h_j_k_one_unit_divided = error_h_j_k_one_unit/(x * learning_rate)
                                error_h_j_k.append(error_h_j_k_one_unit)
                            else:
                                if (len(error_h) == 0):
                                    error_h_j_k_one_unit =calculate_delta_weight(activation_function, learning_rate, target, output, net, weights, x, False, error_k_divided)
                                    error_h_j_k_one_unit_divided = error_h_j_k_one_unit/(x * learning_rate)
                                    error_h_j_k.append(error_h_j_k_one_unit)
                                else:
                                    error_h_j_k_one_unit = calculate_delta_weight(activation_function, learning_rate, target, output, net, weights, x, False, error_h_j_divided[-1])
                                    error_h_j_k_one_unit_divided = error_h_j_k_one_unit/(x * learning_rate)
                                    error_h_j_k.append(error_h_j_k_one_unit)

                        error_h_j.append(error_h_j_k)
                        error_h_j_divided.append(error_h_j_k_one_unit_divided)
                    error_h.append(error_h_j)
                    

                # Update delta weight
                for j in range(len(initial_weights)-1, -1, -1):
                    if j == len(initial_weights)-1:
                        for k in range(len(initial_weights[j])):
                            for l in range(len(initial_weights[j][k])):              
                                delta_weight[j][k][l] += error_k[l][k]
                    else:
                        for k in range(len(initial_weights[j])):
                            for l in range(len(initial_weights[j][k])):
                                delta_weight[j][k][l] += error_h[j][l][k]
                
            # Update weight per batch
            # sum delta weight
            for i in range(len(initial_weights)):
                for j in range(len(initial_weights[i])):
                    for k in range(len(initial_weights[i][j])):
                        initial_weights[i][j][k] += delta_weight[i][j][k]
            
        # Calculate error
        for data in range(input.shape[0]):
            output_forward = forward_propagation(model, input, initial_weights)

            activation_function = model["layers"][-1]["activation_function"]
            if activation_function == "softmax":
                error += loss_function_softmax(target_input[data], output_forward[-1][data])
            else:
                error += loss_function_sse(target_input[data], output_forward[-1][data])

    return initial_weights
                

In [43]:
# Load JSON input
with open("test/backpropagation/linear_small_lr.json", "r") as file:
    data = json.load(file)

# Extract data
model = data["case"]["model"]
input_data = np.array(data["case"]["input"])
initial_weights = [np.array(layer_weights) for layer_weights in data["case"]["initial_weights"]]
target = np.array(data["case"]["target"])
learning_rate = data["case"]["learning_parameters"]["learning_rate"]
batch_size = data["case"]["learning_parameters"]["batch_size"]
max_iteration = data["case"]["learning_parameters"]["max_iteration"]
error_threshold = data["case"]["learning_parameters"]["error_threshold"]
stopped_by = data["expect"]["stopped_by"]
final_weights = [np.array(layer_weights) for layer_weights in data["expect"]["final_weights"]]

new_weight = back_propagation(model, input_data, initial_weights, target, learning_rate, batch_size, stopped_by, max_iteration, error_threshold)
error = 0
for i in range(len(new_weight)):
    for j in range(len(new_weight[i])):
        for k in range(len(new_weight[i][j])):
            error += abs(new_weight[i][j][k] - final_weights[i][j][k])
print(new_weight)
print("Error: ", error)

[array([[ 0.1012,  0.3006,  0.1991],
       [ 0.4024,  0.201 , -0.7019],
       [ 0.1018, -0.799 ,  0.4987]])]
Error:  0.001600000000000018


In [44]:
# Load JSON input
with open("test/backpropagation/linear_two_iteration.json", "r") as file:
    data = json.load(file)

# Extract data
model = data["case"]["model"]
input_data = np.array(data["case"]["input"])
initial_weights = [np.array(layer_weights) for layer_weights in data["case"]["initial_weights"]]
target = np.array(data["case"]["target"])
learning_rate = data["case"]["learning_parameters"]["learning_rate"]
batch_size = data["case"]["learning_parameters"]["batch_size"]
max_iteration = data["case"]["learning_parameters"]["max_iteration"]
error_threshold = data["case"]["learning_parameters"]["error_threshold"]
stopped_by = data["expect"]["stopped_by"]
final_weights = [np.array(layer_weights) for layer_weights in data["expect"]["final_weights"]]

new_weight = back_propagation(model, input_data, initial_weights, target, learning_rate, batch_size, stopped_by, max_iteration, error_threshold)
error = 0
for i in range(len(new_weight)):
    for j in range(len(new_weight[i])):
        for k in range(len(new_weight[i][j])):
            error += abs(new_weight[i][j][k] - final_weights[i][j][k])
print(new_weight)
print("Error: ", error)

[array([[ 0.166,  0.338,  0.153],
       [ 0.502,  0.226, -0.789],
       [ 0.214, -0.718,  0.427]])]
Error:  3.3306690738754696e-16


In [45]:
# Load JSON input
with open("test/backpropagation/linear.json", "r") as file:
    data = json.load(file)

# Extract data
model = data["case"]["model"]
input_data = np.array(data["case"]["input"])
initial_weights = [np.array(layer_weights) for layer_weights in data["case"]["initial_weights"]]
target = np.array(data["case"]["target"])
learning_rate = data["case"]["learning_parameters"]["learning_rate"]
batch_size = data["case"]["learning_parameters"]["batch_size"]
max_iteration = data["case"]["learning_parameters"]["max_iteration"]
error_threshold = data["case"]["learning_parameters"]["error_threshold"]
stopped_by = data["expect"]["stopped_by"]
final_weights = [np.array(layer_weights) for layer_weights in data["expect"]["final_weights"]]

new_weight = back_propagation(model, input_data, initial_weights, target, learning_rate, batch_size, stopped_by, max_iteration, error_threshold)
error = 0
for i in range(len(new_weight)):
    for j in range(len(new_weight[i])):
        for k in range(len(new_weight[i][j])):
            error += abs(new_weight[i][j][k] - final_weights[i][j][k])
print(new_weight)
print("Error: ", error)

[array([[ 0.22,  0.36,  0.11],
       [ 0.64,  0.3 , -0.89],
       [ 0.28, -0.7 ,  0.37]])]
Error:  5.551115123125783e-17


In [46]:
# Load JSON input
with open("test/backpropagation/relu_b.json", "r") as file:
    data = json.load(file)

# Extract data
model = data["case"]["model"]
input_data = np.array(data["case"]["input"])
initial_weights = [np.array(layer_weights) for layer_weights in data["case"]["initial_weights"]]
target = np.array(data["case"]["target"])
learning_rate = data["case"]["learning_parameters"]["learning_rate"]
batch_size = data["case"]["learning_parameters"]["batch_size"]
max_iteration = data["case"]["learning_parameters"]["max_iteration"]
error_threshold = data["case"]["learning_parameters"]["error_threshold"]
stopped_by = data["expect"]["stopped_by"]
final_weights = [np.array(layer_weights) for layer_weights in data["expect"]["final_weights"]]

new_weight = back_propagation(model, input_data, initial_weights, target, learning_rate, batch_size, stopped_by, max_iteration, error_threshold)
error = 0
for i in range(len(new_weight)):
    for j in range(len(new_weight[i])):
        for k in range(len(new_weight[i][j])):
            error += abs(new_weight[i][j][k] - final_weights[i][j][k])
print(new_weight)
print("Error: ", error)

[array([[-0.211 ,  0.105 ,  0.885 ],
       [ 0.3033,  0.5285,  0.3005],
       [-0.489 , -0.905 ,  0.291 ]])]
Error:  8.326672684688674e-17


In [48]:
# Load JSON input
with open("test/backpropagation/sigmoid.json", "r") as file:
    data = json.load(file)

# Extract data
model = data["case"]["model"]
input_data = np.array(data["case"]["input"])
initial_weights = [np.array(layer_weights) for layer_weights in data["case"]["initial_weights"]]
target = np.array(data["case"]["target"])
learning_rate = data["case"]["learning_parameters"]["learning_rate"]
batch_size = data["case"]["learning_parameters"]["batch_size"]
max_iteration = data["case"]["learning_parameters"]["max_iteration"]
error_threshold = data["case"]["learning_parameters"]["error_threshold"]
stopped_by = data["expect"]["stopped_by"]
final_weights = [np.array(layer_weights) for layer_weights in data["expect"]["final_weights"]]

new_weight = back_propagation(model, input_data, initial_weights, target, learning_rate, batch_size, stopped_by, max_iteration, error_threshold)
error = 0
for i in range(len(new_weight)):
    for j in range(len(new_weight[i])):
        for k in range(len(new_weight[i][j])):
            error += abs(new_weight[i][j][k] - final_weights[i][j][k])
print(new_weight)
print("Error: ", error)

[array([[0.23291176, 0.06015346],
       [0.12884088, 0.64849474],
       [0.837615  , 0.23158199]])]
Error:  0.0002978285953069218


C:\Users\altha\AppData\Local\Temp/ipykernel_16856/4051348221.py:103: RuntimeWarning: invalid value encountered in double_scalars
  error_k_j_one_unit_divided = error_k_j_one_unit/(x * learning_rate)


In [49]:
# Load JSON input
with open("test/backpropagation/softmax.json", "r") as file:
    data = json.load(file)

# Extract data
model = data["case"]["model"]
input_data = np.array(data["case"]["input"])
initial_weights = [np.array(layer_weights) for layer_weights in data["case"]["initial_weights"]]
target = np.array(data["case"]["target"])
learning_rate = data["case"]["learning_parameters"]["learning_rate"]
batch_size = data["case"]["learning_parameters"]["batch_size"]
max_iteration = data["case"]["learning_parameters"]["max_iteration"]
error_threshold = data["case"]["learning_parameters"]["error_threshold"]
stopped_by = data["expect"]["stopped_by"]
final_weights = [np.array(layer_weights) for layer_weights in data["expect"]["final_weights"]]

new_weight = back_propagation(model, input_data, initial_weights, target, learning_rate, batch_size, stopped_by, max_iteration, error_threshold)
error = 0
for i in range(len(new_weight)):
    for j in range(len(new_weight[i])):
        for k in range(len(new_weight[i][j])):
            error += abs(new_weight[i][j][k] - final_weights[i][j][k])
print(new_weight)
print("Error: ", error)

[array([[-0.00742189,  0.83363136, -0.22620947],
       [-0.11691988,  0.83776353,  0.33315635],
       [ 0.3723289 , -0.67004192,  0.07671302],
       [ 0.42736295,  0.61379232, -0.33115527],
       [ 0.3777683 ,  0.41675988,  0.53747181],
       [-0.77750271,  0.2954632 ,  0.57403951],
       [-0.70395444, -0.30833046,  0.4552849 ],
       [ 0.55357479,  0.04777754, -0.61835233],
       [ 0.70642187, -0.21360665, -0.14181521]])]
Error:  3.344287527779199


In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

# Load dataset
dataset = pd.read_csv("test/backpropagation/iris.csv")
dataset = dataset.drop(columns=["Id"])
dataset = dataset.sample(frac=1).reset_index(drop=True)
learning_data = dataset.drop(columns=["Species"])
target_data = dataset["Species"]
mlp = MLPClassifier(hidden_layer_sizes=(5,9,3), max_iter=1000, activation='logistic', learning_rate='constant', learning_rate_init=0.01,)
mlp.fit(learning_data, target_data)

y_pred = mlp.predict(learning_data)
print("Predicted values:")
print(y_pred)
print("Actual values:")
print(target_data)
print("Accuracy:", mlp.score(learning_data, target_data))

Predicted values:
['Iris-setosa' 'Iris-virginica' 'Iris-setosa' 'Iris-setosa'
 'Iris-virginica' 'Iris-virginica' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-virginica' 'Iris-versicolor'
 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-virginica' 'Iris-versicolor'
 'Iris-setosa' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor' 'Iris-setosa'
 'Iris-virginica' 'Iris-setosa' 'Iris-virginica' 'Iris-setosa'
 'Iris-versicolor' 'Iris-setosa' 'Iris-versicolor' 'Iris-setosa'
 'Iris-setosa' 'Iris-virginica' 'Iris-virginica' 'Iris-versicolor'
 'Iris-virginica' 'Iris-setosa' 'Iris-virginica' 'Iris-virginica'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-virginica'
 'Iris-virginica' 'Iris-versicolor' 'Iris-virginica' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor'
 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica' 

In [41]:
# Load JSON input
with open("test/backpropagation/iris.json", "r") as file:
    data = json.load(file)

# Extract data
model = data["case"]["model"]
initial_weights = [np.array(layer_weights) for layer_weights in data["case"]["initial_weights"]]
learning_rate = data["case"]["learning_parameters"]["learning_rate"]
batch_size = data["case"]["learning_parameters"]["batch_size"]
max_iteration = data["case"]["learning_parameters"]["max_iteration"]
error_threshold = data["case"]["learning_parameters"]["error_threshold"]
stopped_by = data["expect"]["stopped_by"]
final_weights = [np.array(layer_weights) for layer_weights in data["expect"]["final_weights"]]

# iris with implemented code
input_data = learning_data.to_numpy()
target = []
for i in range(len(target_data)):
    if target_data[i] == "Iris-setosa":
        target.append([1,0,0])
    elif target_data[i] == "Iris-versicolor":
        target.append([0,1,0])
    else:
        target.append([0,0,1])
target = np.array(target)

new_weight = back_propagation(model, input_data, initial_weights, target, learning_rate, batch_size, stopped_by, max_iteration, error_threshold)
print("new weight: ", new_weight)
output = forward_propagation(model, input_data, initial_weights)

count = 0
for i in range(len(output[-1])):
    max = -99999999
    for j in range(3):
        if output[-1][i][j] > max:
            max = output[-1][i][j]
            idx = j
    
    if (idx == 0) : 
        temp = [1,0,0]
    elif (idx == 1):
        temp = [0,1,0]
    else:
        temp = [0,0,1]

    print(temp,target[i])
    if (np.array_equal(temp, target[i])) :
        count += 1
print("Accuracy: ", count/len(output[-1]))

new weight:  [array([[ 0.20231173,  0.09499589, -0.15178272],
       [ 0.15688046, -0.0423218 , -0.56151539],
       [ 0.69338222, -0.39335846, -0.49874664],
       [-1.06830723,  0.25029815,  0.80246466],
       [-0.41899993, -0.03777484,  0.65787204]])]
[1, 0, 0] [1 0 0]
[0, 0, 1] [0 0 1]
[1, 0, 0] [1 0 0]
[1, 0, 0] [1 0 0]
[0, 0, 1] [0 0 1]
[0, 0, 1] [0 0 1]
[1, 0, 0] [1 0 0]
[1, 0, 0] [1 0 0]
[1, 0, 0] [1 0 0]
[1, 0, 0] [1 0 0]
[0, 0, 1] [0 0 1]
[0, 1, 0] [0 1 0]
[1, 0, 0] [1 0 0]
[0, 1, 0] [0 1 0]
[0, 1, 0] [0 1 0]
[1, 0, 0] [1 0 0]
[0, 1, 0] [0 1 0]
[0, 1, 0] [0 1 0]
[0, 0, 1] [0 0 1]
[0, 1, 0] [0 1 0]
[1, 0, 0] [1 0 0]
[0, 0, 1] [0 1 0]
[0, 1, 0] [0 1 0]
[0, 1, 0] [0 1 0]
[0, 0, 1] [0 0 1]
[1, 0, 0] [1 0 0]
[0, 1, 0] [0 1 0]
[1, 0, 0] [1 0 0]
[0, 0, 1] [0 0 1]
[1, 0, 0] [1 0 0]
[0, 0, 1] [0 0 1]
[1, 0, 0] [1 0 0]
[0, 1, 0] [0 1 0]
[1, 0, 0] [1 0 0]
[0, 0, 1] [0 1 0]
[1, 0, 0] [1 0 0]
[1, 0, 0] [1 0 0]
[0, 0, 1] [0 0 1]
[0, 0, 1] [0 0 1]
[0, 1, 0] [0 1 0]
[0, 0, 1] [0 0 1]
[1, 0,

In [ ]:
# function to save model
def save_model(model, input_data, new_weight, stopped_by):
    data = {
        "case": {
            "model": model,
            "input": input_data.tolist(),
            "weights": new_weight[0].tolist()  # Access the NumPy array inside the list
        },
        "expect": {
            "stopped_by": stopped_by,
            "final_weights": new_weight[0].tolist()  # Access the NumPy array inside the list
        }
    }

    with open("test/model.json", "w") as file:
        json.dump(data, file, indent=2)